### Strong baseline : 0.76023
### ---My Best Score : 0.74975---
### Simple baseline : 0.68334

In [1]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

myseed = 1234  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [2]:
print('Loading data ...')

train = np.load('../input/ml2021spring-hw2/timit_11/timit_11/train_11.npy')
train_label = np.load('../input/ml2021spring-hw2/timit_11/timit_11/train_label_11.npy')
test = np.load('../input/ml2021spring-hw2/timit_11/timit_11/test_11.npy').reshape((451552, 11, 39))

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...

Size of training data: (1229932, 429)

Size of testing data: (451552, 11, 39)


In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

In [4]:
VAL_RATIO = 0.1

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]

train_x = train_x.reshape((1106938, 11, 39))
val_x = val_x.reshape((122994, 11, 39))

print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 11, 39)

Size of validation set: (122994, 11, 39)


In [5]:
BATCH_SIZE = 512

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

129

In [7]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(39, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(5632, 1024)
        self.out = nn.Linear(1024, 39) 

        self.dropout = nn.Dropout(0.5)
        self.act_fn = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.act_fn(x)

        x = self.layer2(x)
        x = self.dropout(x)
        x = self.act_fn(x)

        x = torch.flatten(x, 1)
        x = self.layer3(x)
        x = self.dropout(x)
        x = self.act_fn(x)

        x = self.out(x)
        
        return x

In [8]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 200               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.RAdam(model.parameters(),lr=learning_rate, weight_decay=1e-7)

DEVICE: cuda


In [10]:
ES_patience = 10
ES_counter = 0

In [11]:
# start training

best_loss = 1000.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_loss/len(val_loader) < best_loss:
                ES_counter = -1
                best_loss = val_loss/len(val_loader)
                torch.save(model.state_dict(), model_path)
                print('saving model with loss {:.3f}'.format(best_loss))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
    ES_counter += 1  
    if ES_counter == ES_patience:
        print('---Early Stopping---')
        break

[001/200] Train Acc: 0.505274 Loss: 1.680497 | Val Acc: 0.657073 loss: 1.107851

saving model with loss 1.108

[002/200] Train Acc: 0.635395 Loss: 1.160648 | Val Acc: 0.698644 loss: 0.945956

saving model with loss 0.946

[003/200] Train Acc: 0.669523 Loss: 1.036131 | Val Acc: 0.717279 loss: 0.872202

saving model with loss 0.872

[004/200] Train Acc: 0.687897 Loss: 0.968223 | Val Acc: 0.726629 loss: 0.831120

saving model with loss 0.831

[005/200] Train Acc: 0.700777 Loss: 0.922011 | Val Acc: 0.734044 loss: 0.805558

saving model with loss 0.806

[006/200] Train Acc: 0.710426 Loss: 0.886998 | Val Acc: 0.740207 loss: 0.782898

saving model with loss 0.783

[007/200] Train Acc: 0.718019 Loss: 0.859241 | Val Acc: 0.744459 loss: 0.766966

saving model with loss 0.767

[008/200] Train Acc: 0.723907 Loss: 0.837101 | Val Acc: 0.748597 loss: 0.752816

saving model with loss 0.753

[009/200] Train Acc: 0.729597 Loss: 0.816781 | Val Acc: 0.750524 loss: 0.744069

saving model with loss 0.744

[

In [12]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [13]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

In [14]:
with open('submission.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))